# Arithmetic

An important function of pandas is the arithmetic behaviour for objects with different indices. When adding objects, if the index pairs are not equal, the corresponding index in the result will be the union of the index pairs. For users with database experience, this is comparable to an automatic [outer join](https://en.wikipedia.org/wiki/Join_(SQL)#Outer_join) on the index labels. Let’s look at an example:

In [1]:
import numpy as np
import pandas as pd

s1 = pd.Series(np.random.randn(5))
s2 = pd.Series(np.random.randn(7))

If you add these values, you get:

In [2]:
s1 + s2

0    0.207066
1    0.264242
2    0.102994
3    0.700203
4    0.708296
5         NaN
6         NaN
dtype: float64

The internal data matching leads to missing values at the points of the labels that do not overlap. Missing values are then passed on in further arithmetic calculations.

For DataFrames, alignment is performed for both rows and columns:

In [3]:
df1 = pd.DataFrame(np.random.randn(5,3))
df2 = pd.DataFrame(np.random.randn(7,2))

When the two DataFrames are added together, the result is a DataFrame whose index and columns are the unions of those in each of the DataFrames above:

In [4]:
df1 + df2

,0,1,2
0,2.365608,0.016872,NaN
1,3.014176,0.158577,NaN
2,-1.253507,-0.069276,NaN
3,0.966925,1.856066,NaN
4,-2.636667,-1.296551,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


Since column 2 does not appear in both DataFrame objects, its values appear as missing in the result. The same applies to the rows whose labels do not appear in both objects.

## Arithmetic methods with fill values

In arithmetic operations between differently indexed objects, a special value (e.g. `0`) can be useful if an axis label is found in one object but not in the other.  The `add` method can pass the `fill_value` argument:

In [5]:
df12 = df1.add(df2, fill_value=0)

df12

,0,1,2
0,2.365608,0.016872,-0.184961
1,3.014176,0.158577,-0.212261
2,-1.253507,-0.069276,-0.660886
3,0.966925,1.856066,-0.421662
4,-2.636667,-1.296551,1.059397
5,0.290635,-0.110048,NaN
6,0.134017,-0.301216,NaN


In the following example, we set the two remaining NaN values to `0`:

In [6]:
df12.iloc[[5,6], [2]] = 0

In [7]:
df12

,0,1,2
0,2.365608,0.016872,-0.184961
1,3.014176,0.158577,-0.212261
2,-1.253507,-0.069276,-0.660886
3,0.966925,1.856066,-0.421662
4,-2.636667,-1.296551,1.059397
5,0.290635,-0.110048,0.000000
6,0.134017,-0.301216,0.000000


## Arithmetic methods

Method | Description
:----- | :----------
`add`, `radd` | methods for addition (`+`)
`sub`, `rsub` | methods for subtraction (`-`)
`div`, `rdiv` | methods for division (`/`)
`floordiv`, `rfloordiv` | methods for floor division (`//`)
`mul`, `rmul` | methods for multiplication (`*`)
`pow`, `rpow` | methods for exponentiation (`**`)

`r` (English: _reverse_) reverses the method.

## Operations between DataFrame and Series

As with NumPy arrays of different dimensions, the arithmetic between DataFrame and Series is also defined.

In [8]:
s1 + df12

,0,1,2,3,4
0,1.550229,0.355749,-0.039186,NaN,NaN
1,2.198798,0.497454,-0.066486,NaN,NaN
2,-2.068886,0.269600,-0.515111,NaN,NaN
3,0.151546,2.194942,-0.275887,NaN,NaN
4,-3.452046,-0.957675,1.205172,NaN,NaN
5,-0.524743,0.228829,0.145775,NaN,NaN
6,-0.681361,0.037660,0.145775,NaN,NaN


If we add `s1` with `df12`, the additon is done once for each line. This is called _broadcasting_. By default, the arithmetic between the DataFrame and the series corresponds to the index of the series in the columns of the DataFrame, with the rows being broadcast down.

If an index value is found neither in the columns of the DataFrame nor in the index of the series, the objects are re-indexed to form the union:

If instead you want to transfer the columns and match the rows, you must use one of the arithmetic methods, for example:

In [9]:
df12.add(s2, axis='index')

,0,1,2
0,3.388052,1.039317,0.837483
1,2.939542,0.083943,-0.286895
2,-1.296288,-0.112057,-0.703668
3,-0.905759,-0.016618,-2.294346
4,-2.579514,-1.239397,1.116551
5,-0.345143,-0.745827,-0.635779
6,0.198245,-0.236989,0.064228


The axis number you pass is the axis to be aligned to. In this case, the row index of the DataFrame (`axis='index'` or `axis=0`) is to be adjusted and transmitted.

## Function application and mapping

`numpy.ufunc` (element-wise array methods) also work with pandas objects:

In [10]:
np.abs(df12)

,0,1,2
0,2.365608,0.016872,0.184961
1,3.014176,0.158577,0.212261
2,1.253507,0.069276,0.660886
3,0.966925,1.856066,0.421662
4,2.636667,1.296551,1.059397
5,0.290635,0.110048,0.000000
6,0.134017,0.301216,0.000000


Another common operation is to apply a function to one-dimensional arrays on each column or row. The [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method does just that:

In [11]:
df12

,0,1,2
0,2.365608,0.016872,-0.184961
1,3.014176,0.158577,-0.212261
2,-1.253507,-0.069276,-0.660886
3,0.966925,1.856066,-0.421662
4,-2.636667,-1.296551,1.059397
5,0.290635,-0.110048,0.000000
6,0.134017,-0.301216,0.000000


In [12]:
f = lambda x: x.max() - x.min()

df12.apply(f)

0    5.650844
1    3.152617
2    1.720283
dtype: float64

Here the function `f`, which calculates the difference between the maximum and minimum of a row, is called once for each column of the frame. The result is a row with the columns of the frame as index.

If you pass `axis='columns'` to `apply`, the function will be called once per line instead:

In [13]:
df12.apply(f, axis='columns')

0    2.550569
1    3.226437
2    1.184231
3    2.277728
4    3.696064
5    0.400683
6    0.435233
dtype: float64

Many of the most common array statistics (such as `sum` and `mean`) are DataFrame methods, so the use of `apply` is not necessary.

The function passed to apply does not have to return a single value; it can also return a series with multiple values:

In [14]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

df12.apply(f)

,0,1,2
min,-2.636667,-1.296551,-0.660886
max,3.014176,1.856066,1.059397


You can also use element-wise Python functions. Suppose you want to round each floating point value in `df12` to two decimal places, you can do this with [pandas.DataFrame.applymap](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html):

In [15]:
f = lambda x: round(x, 2)

df12.applymap(f)

,0,1,2
0,2.37,0.02,-0.18
1,3.01,0.16,-0.21
2,-1.25,-0.07,-0.66
3,0.97,1.86,-0.42
4,-2.64,-1.30,1.06
5,0.29,-0.11,0.00
6,0.13,-0.30,0.00


The reason for the name `applymap` is that Series has a `map` method for applying an element-wise function:

In [16]:
df12[2].map(f)

0   -0.18
1   -0.21
2   -0.66
3   -0.42
4    1.06
5    0.00
6    0.00
Name: 2, dtype: float64